In [2]:
import os
import json
import time
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

import pandas as pd
from transformers import pipeline
from transformers import BertTokenizer, BertForQuestionAnswering
from transformers import AlbertTokenizer, AlbertForQuestionAnswering
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from transformers.data.processors.squad import (
    SquadResult, 
    SquadV1Processor, 
    SquadV2Processor, 
    SquadExample, 
    SquadFeatures,
)
from transformers.data.processors.utils import DataProcessor, InputExample
from transformers import (
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    squad_convert_examples_to_features,
)
from transformers.data.metrics.squad_metrics import (
    compute_predictions_logits,
    squad_evaluate,
)

In [3]:
# os.path.abspath(os.getcwd())
data_dir = os.path.abspath('../../qary/data')
path = os.path.abspath('../../qary/data/squad')

In [4]:
modelnames = [
    "bert-large-uncased-whole-word-masking-finetuned-squad",
    "ktrapeznikov/albert-xlarge-v2-squad-v2",
    "mrm8488/bert-tiny-5-finetuned-squadv2",
    "twmkn9/albert-base-v2-squad2",
    'distilbert-base-uncased',
    "distilbert-base-cased-distilled-squad",
]

In [5]:
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = AutoModelForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [6]:
processor = SquadV2Processor()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12,

In [7]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

Convert SQuAD examples to features using processors:

In [8]:
examples = processor.get_dev_examples(path)

100%|███████████████████████████████████████████████████████████████████████████████| 35/35 [00:12<00:00,  2.84it/s]


In [9]:
features, dataset = squad_convert_examples_to_features(
    examples=examples,
    tokenizer=tokenizer,
    max_seq_length=512,
    doc_stride = 128,
    max_query_length=256,
    is_training=False,
    return_dataset='pt',
    threads=4,
)

add example index and unique id: 100%|████████████████████████████████████| 11873/11873 [00:00<00:00, 351633.02it/s]


In [10]:
eval_sampler = SequentialSampler(dataset)
eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

In [ ]:
all_results = []

for batch in tqdm(eval_dataloader):
    model.eval()
    batch = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2],
        }

        example_indices = batch[3]

        outputs = model(**inputs)

        for i, example_index in enumerate(example_indices):
            eval_feature = features[example_index.item()]
            unique_id = int(eval_feature.unique_id)

            output = [to_list(output[i]) for output in outputs]

            start_logits, end_logits = output
            result = SquadResult(unique_id, start_logits, end_logits)
            all_results.append(result)

In [187]:
import warnings
warnings.filterwarnings('ignore')

In [188]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

UsageError: Line magic function `%install_ext` not found.


In [ ]:
output_prediction_file = "predictions.json"
output_nbest_file = "nbest_predictions.json"
output_null_log_odds_file = "null_predictions.json"
n_best_size = 1
max_answer_length = 254
do_lower_case = True
null_score_diff_threshold = 0.5


predictions = compute_predictions_logits(
    examples,
    features,
    all_results,
    n_best_size,
    max_answer_length,
    do_lower_case,
    output_prediction_file,
    output_nbest_file,
    output_null_log_odds_file,
    False,  # verbose_logging
    True,  # version_2_with_negative
    null_score_diff_threshold,
    tokenizer,
)

In [ ]:
null_odds = json.load(open("null_predictions.json", 'rb'))

In [ ]:
best_f1_thresh = -21.030719757080078
best_exact_thresh = -21.529870867729187

In [ ]:
results = squad_evaluate(examples, 
                         predictions,
                         no_answer_probs=null_odds, 
                         no_answer_probability_threshold=best_f1_thresh)

In [12]:
preds_v1_path = os.path.abspath('../../qary/data/predictions-v1')
preds_v2_path = os.path.abspath('../../qary/data/predictions-v2')

In [13]:
processor_v1 = SquadV1Processor()
examples_v1 = processor_v1.get_dev_examples(path)

100%|███████████████████████████████████████████████████████████████████████████████| 48/48 [00:12<00:00,  3.93it/s]


In [14]:
processor_v2 = SquadV2Processor()
examples_v2 = processor_v2.get_dev_examples(path)

100%|███████████████████████████████████████████████████████████████████████████████| 35/35 [00:11<00:00,  3.07it/s]


In [18]:
reports_v1 = []
for m in modelnames:
    report = {}
    modelname=m.replace('/', '-')
    mpath = os.path.join(preds_v1_path, modelname)
    mpreds_path = os.path.join(mpath, "predictions.json")
    predictions = json.load(open(mpreds_path , 'rb'))
    report = dict(squad_evaluate(examples_v1, 
                             predictions,
                            ))
    
    report['MODEL'] = m
    reports_v1.append(report)

In [34]:
reports_v1_df = pd.DataFrame(reports_v1)

In [28]:
reports_v2 = []
for m in modelnames:
    report = {}
    modelname=m.replace('/', '-')
    print(modelname)
    mpath = os.path.join(preds_v2_path, modelname)
    mpreds_path = os.path.join(mpath, "predictions.json")
    mnodds_path = os.path.join(mpath, "null_predictions.json")
    predictions = json.load(open(mpreds_path , 'rb'))
    null_odds = json.load(open(mnodds_path, 'rb'))
    report = dict(squad_evaluate(examples_v2, 
                                 predictions, 
                                 no_answer_probs=null_odds, 
                                 no_answer_probability_threshold=0.5))
    
    report['MODEL'] = m
    reports_v2.append(report)

bert-large-uncased-whole-word-masking-finetuned-squad
ktrapeznikov-albert-xlarge-v2-squad-v2
mrm8488-bert-tiny-5-finetuned-squadv2
twmkn9-albert-base-v2-squad2
distilbert-base-uncased
distilbert-base-cased-distilled-squad


In [31]:
reports_v2_df = pd.DataFrame(reports_v2)

In [56]:
def compact_dataframe(df):
    columns_to_drop = [col for col 
                       in list(df.columns) 
                       if "total" in col]
#     print(df.round(2).set_index('MODEL').drop(columns = columns_to_drop))
    return df.round(2).set_index('MODEL').drop(columns = columns_to_drop).sort_values(by=['exact', 'f1'], ascending=False)

In [57]:
compact_v1_df = compact_dataframe(reports_v1_df)
compact_v2_df = compact_dataframe(reports_v2_df)

In [58]:
compact_v1_df

,exact,f1,HasAns_exact,HasAns_f1,best_exact,best_exact_thresh,best_f1,best_f1_thresh
MODEL,,,,,,,,
bert-large-uncased-whole-word-masking-finetuned-squad,86.29,92.47,86.29,92.47,86.29,0.0,92.47,0.0
ktrapeznikov/albert-xlarge-v2-squad-v2,81.49,87.14,81.49,87.14,81.49,0.0,87.14,0.0
distilbert-base-cased-distilled-squad,78.28,85.70,78.28,85.70,78.28,0.0,85.70,0.0
twmkn9/albert-base-v2-squad2,74.04,80.59,74.04,80.59,74.04,0.0,80.59,0.0
mrm8488/bert-tiny-5-finetuned-squadv2,50.82,57.71,50.82,57.71,50.82,0.0,57.71,0.0
distilbert-base-uncased,0.03,1.16,0.03,1.16,0.03,0.0,1.16,0.0


In [59]:
compact_v2_df

,exact,f1,HasAns_exact,HasAns_f1,NoAns_exact,NoAns_f1,best_exact,best_exact_thresh,best_f1,best_f1_thresh
MODEL,,,,,,,,,,
ktrapeznikov/albert-xlarge-v2-squad-v2,84.46,87.47,80.01,86.04,88.90,88.90,84.71,-1.31,87.68,-1.22
twmkn9/albert-base-v2-squad2,77.92,81.38,72.84,79.78,82.98,82.98,78.38,-3.34,81.73,-1.82
mrm8488/bert-tiny-5-finetuned-squadv2,57.85,61.24,50.10,56.89,65.57,65.57,59.57,-2.07,62.20,-1.87
bert-large-uncased-whole-word-masking-finetuned-squad,42.63,45.93,85.24,91.85,0.13,0.13,65.83,-21.53,67.13,-21.03
distilbert-base-cased-distilled-squad,39.02,42.86,78.10,85.78,0.05,0.05,55.61,-22.67,56.85,-22.42
distilbert-base-uncased,0.00,0.61,0.00,1.23,0.00,0.00,50.07,0.00,50.07,-1.60


In [71]:
reports_v2_adjusted = []
for m in modelnames:
    report = {}
    modelname=m.replace('/', '-')
    print(modelname)
    mpath = os.path.join(preds_v2_path, modelname)
    mpreds_path = os.path.join(mpath, "predictions.json")
    mnodds_path = os.path.join(mpath, "null_predictions.json")
    predictions = json.load(open(mpreds_path , 'rb'))
    null_odds = json.load(open(mnodds_path, 'rb'))
    report_raw = dict(squad_evaluate(examples_v2, 
                                 predictions, 
                                 no_answer_probs=null_odds, 
                                 no_answer_probability_threshold=0.5))
    
    best_thresh = report_raw['best_exact_thresh']
    
    report = dict(squad_evaluate(examples_v2, 
                             predictions, 
                             no_answer_probs=null_odds, 
                             no_answer_probability_threshold=best_thresh ))
    
    report['MODEL'] = m
    reports_v2_adjusted.append(report)

bert-large-uncased-whole-word-masking-finetuned-squad
ktrapeznikov-albert-xlarge-v2-squad-v2
mrm8488-bert-tiny-5-finetuned-squadv2
twmkn9-albert-base-v2-squad2
distilbert-base-uncased
distilbert-base-cased-distilled-squad


In [72]:
reports_v2_adj_df = compact_dataframe(pd.DataFrame(reports_v2_adjusted))

In [73]:
reports_v2_adj_df

,exact,f1,HasAns_exact,HasAns_f1,NoAns_exact,NoAns_f1,best_exact,best_exact_thresh,best_f1,best_f1_thresh
MODEL,,,,,,,,,,
ktrapeznikov/albert-xlarge-v2-squad-v2,84.71,87.67,79.10,85.02,90.31,90.31,84.71,-1.31,87.68,-1.22
twmkn9/albert-base-v2-squad2,78.38,81.57,69.55,75.95,87.18,87.18,78.38,-3.34,81.73,-1.82
bert-large-uncased-whole-word-masking-finetuned-squad,65.83,67.01,58.72,61.09,72.92,72.92,65.83,-21.53,67.13,-21.03
mrm8488/bert-tiny-5-finetuned-squadv2,59.57,62.13,41.70,46.82,77.39,77.39,59.57,-2.07,62.20,-1.87
distilbert-base-cased-distilled-squad,55.61,56.73,42.88,45.13,68.29,68.29,55.61,-22.67,56.85,-22.42
distilbert-base-uncased,6.23,6.85,0.00,1.23,12.45,12.45,50.07,0.00,50.07,-1.60


In [ ]:
reports_v1_adjusted = []
for m in modelnames:
    report = {}
    modelname=m.replace('/', '-')
    print(modelname)
    mpath = os.path.join(preds_v1_path, modelname)
    mpreds_path = os.path.join(mpath, "predictions.json")
    mnodds_path = os.path.join(mpath, "null_predictions.json")
    predictions = json.load(open(mpreds_path , 'rb'))
    null_odds = json.load(open(mnodds_path, 'rb'))
    report_raw = dict(squad_evaluate(examples_v1, 
                                 predictions, 
                                 no_answer_probs=null_odds, 
                                 no_answer_probability_threshold=0.5))
    
    best_thresh = report_raw['best_exact_thresh']
    
#     report = dict(squad_evaluate(examples_v1, 
#                              predictions, 
#                              no_answer_probs=null_odds, 
#                              no_answer_probability_threshold=best_thresh ))
    
    report['MODEL'] = m
    reports_v1_adjusted.append(report)

In [222]:
keep_cols = ['exact', 'f1']
# keep_cols = ['exact', 'f1', 'HasAns_exact', 'HasAns_f1']
all_reports = compact_v1_df[keep_cols].merge(compact_v2_df[keep_cols], left_on=index, right_on=index, suffixes=('_v1', '_v2'))
all_reports = all_reports.rename(columns = {'key_0':'MODEL'})
# all_reports.set_index('MODEL')

In [227]:
all_reports

,MODEL,exact_v1,f1_v1,exact_v2,f1_v2
0,ktrapeznikov/albert-xlarge-v2-squad-v2,86.29,92.47,84.46,87.47
1,twmkn9/albert-base-v2-squad2,81.49,87.14,77.92,81.38
2,bert-large-uncased-whole-word-masking-finetune...,78.28,85.70,57.85,61.24
3,mrm8488/bert-tiny-5-finetuned-squadv2,74.04,80.59,42.63,45.93
4,distilbert-base-cased-distilled-squad,50.82,57.71,39.02,42.86
5,distilbert-base-uncased,0.03,1.16,0.00,0.61


In [232]:
human = {'MODEL': 'human performance', 'exact_v1':86.831, 'f1_v1': 89.452}
all_reports = all_reports.append(human, ignore_index=True).sort_values('exact_v1', ascending = False)


In [233]:
all_reports

,MODEL,exact_v1,f1_v1,exact_v2,f1_v2
6,human performance,86.831,89.452,NaN,NaN
0,ktrapeznikov/albert-xlarge-v2-squad-v2,86.290,92.470,84.46,87.47
1,twmkn9/albert-base-v2-squad2,81.490,87.140,77.92,81.38
2,bert-large-uncased-whole-word-masking-finetune...,78.280,85.700,57.85,61.24
3,mrm8488/bert-tiny-5-finetuned-squadv2,74.040,80.590,42.63,45.93
4,distilbert-base-cased-distilled-squad,50.820,57.710,39.02,42.86
5,distilbert-base-uncased,0.030,1.160,0.00,0.61


In [238]:
compact_v2_df[['exact', 'f1', 'HasAns_exact', 'HasAns_f1', 'NoAns_exact', 'NoAns_f1']]

,exact,f1,HasAns_exact,HasAns_f1,NoAns_exact,NoAns_f1
MODEL,,,,,,
ktrapeznikov/albert-xlarge-v2-squad-v2,84.46,87.47,80.01,86.04,88.90,88.90
twmkn9/albert-base-v2-squad2,77.92,81.38,72.84,79.78,82.98,82.98
mrm8488/bert-tiny-5-finetuned-squadv2,57.85,61.24,50.10,56.89,65.57,65.57
bert-large-uncased-whole-word-masking-finetuned-squad,42.63,45.93,85.24,91.85,0.13,0.13
distilbert-base-cased-distilled-squad,39.02,42.86,78.10,85.78,0.05,0.05
distilbert-base-uncased,0.00,0.61,0.00,1.23,0.00,0.00


In [242]:
keep_cols_v1 = ['HasAns_exact', 'HasAns_f1']
keep_cols_v2 = ['HasAns_exact', 'HasAns_f1', 'NoAns_exact', 'NoAns_f1']
reports_unpack = compact_v2_df[keep_cols_v2].merge(reports_v2_adj_df[keep_cols_v2], left_on=index, right_on=index, suffixes=('', '_ADJ'))
reports_unpack = reports_unpack.rename(columns = {'key_0':'MODEL'}).reset_index(drop=True)
reports_unpack

,MODEL,HasAns_exact,HasAns_f1,NoAns_exact,NoAns_f1,HasAns_exact_ADJ,HasAns_f1_ADJ,NoAns_exact_ADJ,NoAns_f1_ADJ
0,ktrapeznikov/albert-xlarge-v2-squad-v2,80.01,86.04,88.90,88.90,79.10,85.02,90.31,90.31
1,twmkn9/albert-base-v2-squad2,72.84,79.78,82.98,82.98,69.55,75.95,87.18,87.18
2,bert-large-uncased-whole-word-masking-finetune...,50.10,56.89,65.57,65.57,58.72,61.09,72.92,72.92
3,mrm8488/bert-tiny-5-finetuned-squadv2,85.24,91.85,0.13,0.13,41.70,46.82,77.39,77.39
4,distilbert-base-cased-distilled-squad,78.10,85.78,0.05,0.05,42.88,45.13,68.29,68.29
5,distilbert-base-uncased,0.00,1.23,0.00,0.00,0.00,1.23,12.45,12.45


In [170]:
keep_cols = ['HasAns_exact', 'HasAns_f1', 'NoAns_exact', 'NoAns_f1']
v2_vs_adjusted = compact_v2_df[keep_cols].merge(reports_v2_adj_df[keep_cols], left_on=index, right_on=index, suffixes=('', '(ADJ)'))
v2_vs_adjusted = v2_vs_adjusted.rename(columns = {'key_0':'MODEL'}).reset_index(drop=True)
v2_vs_adjusted.sort_values(by=['HasAns_exact', 'HasAns_f1'], ascending = False)

,MODEL,HasAns_exact,HasAns_f1,NoAns_exact,NoAns_f1,HasAns_exact(ADJ),HasAns_f1(ADJ),NoAns_exact(ADJ),NoAns_f1(ADJ)
3,mrm8488/bert-tiny-5-finetuned-squadv2,85.24,91.85,0.13,0.13,41.70,46.82,77.39,77.39
0,ktrapeznikov/albert-xlarge-v2-squad-v2,80.01,86.04,88.90,88.90,79.10,85.02,90.31,90.31
4,distilbert-base-cased-distilled-squad,78.10,85.78,0.05,0.05,42.88,45.13,68.29,68.29
1,twmkn9/albert-base-v2-squad2,72.84,79.78,82.98,82.98,69.55,75.95,87.18,87.18
2,bert-large-uncased-whole-word-masking-finetune...,50.10,56.89,65.57,65.57,58.72,61.09,72.92,72.92
5,distilbert-base-uncased,0.00,1.23,0.00,0.00,0.00,1.23,12.45,12.45


Default model uses distilbert-base-cased-distilled-squad

In [213]:
context = '''
Computational linguistics is often grouped within the field of artificial intelligence 
but was present before the development of artificial intelligence. 
Computational linguistics originated with efforts in the United States in the 1950s to use computers to 
automatically translate texts from foreign languages, particularly Russian scientific journals, into English.[3]
Since computers can make arithmetic (systematic) calculations much faster and more accurately than humans, 
it was thought to be only a short matter of time before they could also begin to process language.[4] 
Computational and quantitative methods are also used historically in the attempted reconstruction of earlier
forms of modern languages and sub-grouping modern languages into language families. 
Earlier methods, such as lexicostatistics and glottochronology, have been proven to be premature and inaccurate. 
However, recent interdisciplinary studies that borrow concepts from biological studies, especially gene mapping, 
have proved to produce more sophisticated analytical tools and more reliable results.[5]
'''

questions=['When was computational linguistics invented?',
          'Which problems computational linguistics is trying to solve?',
          'What methods existed before the emergence of computational linguistics?',
          'Who invented computational linguistics?',
          'Who invented gene mapping?']

In [214]:
for model in modelnames:
    # Sentiment analysis pipeline
    nlp = pipeline('question-answering', model=model, tokenizer=model)
    print(f'Model: {model}')
    print('-----------------')
    for q in questions:
        print(f'Question: {q}')
        result = nlp(context=context, question=q, handle_impossible_answer=True)
        answer = result['answer']
        conf = result['score']
        print(f'Answer: {answer} (confidence score {conf})')
        print(' ')


Model: bert-large-uncased-whole-word-masking-finetuned-squad
-----------------
Question: When was computational linguistics invented?
Answer: 1950s (confidence score 0.7105585285134239)
 
Question: Which problems computational linguistics is trying to solve?
Answer: earlier forms of modern languages and sub-grouping modern languages into language families. (confidence score 0.034796690637104444)
 
Question: What methods existed before the emergence of computational linguistics?
Answer: lexicostatistics and glottochronology, (confidence score 0.8949566496998465)
 
Question: Who invented computational linguistics?
Answer: United States (confidence score 0.5333964470000865)
 
Question: Who invented gene mapping?
Answer: biological studies, (confidence score 0.02638426599066701)
 
Model: ktrapeznikov/albert-xlarge-v2-squad-v2
-----------------
Question: When was computational linguistics invented?
Answer: 1950s (confidence score 0.6412413898187204)
 
Question: Which problems computational


Model: distilbert-base-uncased
-----------------
Question: When was computational linguistics invented?
Answer: English.[3] Since computers can make arithmetic (systematic) (confidence score 1.582584644544934e-05)
 
Question: Which problems computational linguistics is trying to solve?
Answer: into language families. (confidence score 1.5566947074668424e-05)
 
Question: What methods existed before the emergence of computational linguistics?
Answer: into language families. (confidence score 1.5719257432964353e-05)
 
Question: Who invented computational linguistics?
Answer: English.[3] Since computers can make arithmetic (systematic) (confidence score 1.5842987051530207e-05)
 
Question: Who invented gene mapping?
Answer: English.[3] Since computers can make arithmetic (systematic) (confidence score 1.5883810795251857e-05)
 



Model: distilbert-base-cased-distilled-squad
-----------------
Question: When was computational linguistics invented?
Answer: 1950s (confidence score 0.7759537003546768)
 
Question: Which problems computational linguistics is trying to solve?
Answer: gene mapping, (confidence score 0.4235580072416312)
 
Question: What methods existed before the emergence of computational linguistics?
Answer: lexicostatistics and glottochronology, (confidence score 0.8573431178602817)
 
Question: Who invented computational linguistics?
Answer: computers (confidence score 0.7313878935375229)
 
Question: Who invented gene mapping?
Answer: biological studies, (confidence score 0.4788379586462099)
 


In [215]:
for model in modelnames:
    # Sentiment analysis pipeline
    nlp = pipeline('question-answering', model=model, tokenizer=model)
    print(f'Model: {model}')
    print('-----------------')
    for q in questions:
        print(f'Question: {q}')
        result = nlp(context=context, question=q, handle_impossible_answer=False)
        answer = result['answer']
        conf = result['score']
        print(f'Answer: {answer} (confidence score {conf})')
        print(' ')


Model: bert-large-uncased-whole-word-masking-finetuned-squad
-----------------
Question: When was computational linguistics invented?
Answer: 1950s (confidence score 0.7105585285134239)
 
Question: Which problems computational linguistics is trying to solve?
Answer: earlier forms of modern languages and sub-grouping modern languages into language families. (confidence score 0.034796690637104444)
 
Question: What methods existed before the emergence of computational linguistics?
Answer: lexicostatistics and glottochronology, (confidence score 0.8949566496998465)
 
Question: Who invented computational linguistics?
Answer: United States (confidence score 0.5333964470000865)
 
Question: Who invented gene mapping?
Answer: biological studies, (confidence score 0.02638426599066701)
 
Model: ktrapeznikov/albert-xlarge-v2-squad-v2
-----------------
Question: When was computational linguistics invented?
Answer: 1950s (confidence score 0.6412413898187204)
 
Question: Which problems computational


Model: distilbert-base-uncased
-----------------
Question: When was computational linguistics invented?
Answer: languages and sub-grouping modern languages into language families. (confidence score 1.6736800661229642e-05)
 
Question: Which problems computational linguistics is trying to solve?
Answer: languages and sub-grouping modern languages into language families. (confidence score 1.6955155309750225e-05)
 
Question: What methods existed before the emergence of computational linguistics?
Answer: languages and sub-grouping modern languages into language families. (confidence score 1.6916083581960213e-05)
 
Question: Who invented computational linguistics?
Answer: languages and sub-grouping modern languages into language families. (confidence score 1.6536293692165794e-05)
 
Question: Who invented gene mapping?
Answer: languages and sub-grouping modern languages into language families. (confidence score 1.6527565861201984e-05)
 



Model: distilbert-base-cased-distilled-squad
-----------------
Question: When was computational linguistics invented?
Answer: 1950s (confidence score 0.7759537003546768)
 
Question: Which problems computational linguistics is trying to solve?
Answer: gene mapping, (confidence score 0.4235580072416312)
 
Question: What methods existed before the emergence of computational linguistics?
Answer: lexicostatistics and glottochronology, (confidence score 0.8573431178602817)
 
Question: Who invented computational linguistics?
Answer: computers (confidence score 0.7313878935375229)
 
Question: Who invented gene mapping?
Answer: biological studies, (confidence score 0.4788379586462099)
 


In [217]:
from transformers import pipeline

context = '''
Stan Lee[1] (born Stanley Martin Lieber /ˈliːbər/; December 28, 1922 – November 12, 2018) was an American 
comic book writer, editor, publisher, and producer. He rose through the ranks of a family-run business to become 
Marvel Comics' primary creative leader for two decades, leading its expansion from a small division of a publishing 
house to a multimedia corporation that dominated the comics industry. 
'''
nlp = pipeline('question-answering')
result = nlp(context=context, question="Who is Stan Lee?")

In [218]:
result

{'score': 0.2854291316652837,
 'start': 95,
 'end': 159,
 'answer': 'an American comic book writer, editor, publisher, and producer.'}